## 言語処理100本ノック2020 in Haskell

### chapter 1

In [1]:
:extension OverloadedStrings
import qualified Data.Text as T
import Prelude as P

In [2]:
-- 00
src ="stressed" :: T.Text
putStrLn $ T.unpack $ T.reverse src 

desserts

In [3]:
-- 01
src = "パタトクカシーー" :: T.Text

f :: T.Text -> Maybe T.Text
f s | T.length s < 7 =  Nothing
    | otherwise = Just $ T.pack [T.head s, T.index s 2, T.index s 4, T.index s 6]

case f src of 
    Just s -> putStrLn $ T.unpack s
    Nothing -> return ()

パトカー

In [4]:
-- 02
src1 = "パトカー"
src2 = "タクシー"

f :: T.Text -> T.Text -> T.Text
f z w = P.foldr (\s t -> T.pack [fst s, snd s] <> t)  "" (T.zip z w) 

putStrLn $ T.unpack $ f src1 src2

パタトクカシーー

In [5]:
-- 03
src = "Now I need a drink, alcoholic of course, after the heavy lectures involving quantum mechanics."

getCountList :: T.Text -> [Int]
getCountList s = 
    let 
        ws' = P.map (T.filter (\c -> c /= ',' && c /= '.')) (T.words s)
    in 
        builder ws' []
    where 
        builder :: [T.Text] -> [Int] -> [Int]
        builder ws counts | P.null ws = counts
                          | otherwise = builder (P.tail ws) (counts <> [T.length (P.head ws)])

getCountList src

[3,1,4,1,5,9,2,6,5,3,5,8,9,7,9]

In [6]:
-- 04
import qualified Data.HashMap.Strict as HM
import Data.Function
src = "Hi He Lied Because Boron Could Not Oxidize Fluorine. New Nations Might Also Sign Peace Security Clause. Arthur King Can."
onlyOne = [0, 4, 5, 6, 7, 8, 14, 15, 18]

f :: T.Text -> T.Text
f s =
    let
        ws = T.words s
        wns = P.zip [0..(P.length ws -1)] ws
    in seq ws $
        if P.length ws < 19 then ""
        else seq wns $
            P.foldr (\s t -> let (ind, text) = s in if ind `elem` onlyOne then T.pack [T.head text] <> t else T.pack [T.head text, T.index text 1] <> t) "" wns

putStrLn $ T.unpack $ f src

pairingIndex :: T.Text -> HM.HashMap T.Text Int
pairingIndex sentences =
    let
        ws' = T.words src
        ws = P.filter ( /= ".") $! ws'
        wns = P.zip [0..(P.length ws -1)] ws
    in
        flip fix (HM.empty, wns) $ \loop (dict, rest) ->
            if P.null rest then dict
            else
                let (v, k) = P.head rest in
                loop (HM.insert k v dict, P.tail rest)


print $ pairingIndex src 


HHeLiBeBCNOFNeNaMiAlSiPSClArKCa

fromList [("Might",11),("Can.",19),("Not",6),("Nations",10),("Boron",4),("New",9),("Sign",13),("Clause.",16),("Fluorine.",8),("Could",5),("Oxidize",7),("Lied",2),("He",1),("Because",3),("Hi",0),("Also",12),("Peace",14),("Arthur",17),("King",18),("Security",15)]

In [7]:
--05
getNGram :: Int -> T.Text -> [T.Text] -> [T.Text]
getNGram n s nGrams | n <= 0 = nGrams
                    | T.length s < n = nGrams
                    | otherwise = getNGram n (T.drop 1 s) (nGrams <> [T.take n s])

getNWordsGram :: Int -> T.Text -> [T.Text]
getNWordsGram n s = builder n (T.words s) []
    where
        builder :: Int -> [T.Text] -> [T.Text] -> [T.Text]
        builder n ws nGrams | P.length ws < n = nGrams
                            | n <= 0 = nGrams
                            | otherwise = builder n (P.drop 1 ws) (nGrams <> [T.concat (P.map (\t -> t <> " ") $ P.take n ws)])

print $ getNGram 2 "I am an NLPer" []
print $ getNWordsGram 2 "I am an NLPer"

Line 13: Avoid lambda using `infix`
Found:
(\ t -> t <> " ")
Why not:
(<> " ")

["I "," a","am","m "," a","an","n "," N","NL","LP","Pe","er"]

["I am ","am an ","an NLPer "]

In [8]:
-- 06
import qualified Data.HashSet as HS
setX = HS.fromList $ getNGram 2 "paraparaparadise" []
setY = HS.fromList $ getNGram 2 "paragraph" []

print $ HS.unions [setX, setY]
print $ HS.intersection setX setY
print $ HS.difference setX setY

fromList ["ad","gr","ar","pa","di","ag","se","is","ph","ra","ap"]

fromList ["ar","pa","ra","ap"]

fromList ["ad","di","se","is"]

In [10]:
-- 07
{-# Language ScopedTypeVariables #-}
{-# Language TypeApplications #-}
{-# Language ConstraintKinds #-}
{-# Language GADTs #-}

import qualified Type.Reflection as TRF

type Generable a = (Eq a, TRF.Typeable a, Show a)

packToText :: forall a. (Eq a, TRF.Typeable a, Show a) => a -> T.Text
packToText x =  
    case TRF.eqTypeRep (TRF.typeRep @a) (TRF.typeRep @String) of
        -- コンパイラは a と String が等しいことがわかる
        Just TRF.HRefl -> T.pack x
        Nothing -> T.pack $ show x

f :: (Generable a, Generable b, Generable c) => a -> b -> c -> T.Text
f x y z = packToText x <> "時の" <> packToText y <> "は" <> packToText z

putStrLn $ T.unpack $ f 12 "気温" 22.4


12時の気温は22.4

In [11]:
-- 08
import qualified Data.ByteString as BS
import qualified Data.ByteString.UTF8 as BSU
import Data.Char

encryptChar :: Char -> Char
encryptChar c | isLower c = 
    let 
        cb = head $ BS.unpack $ BSU.fromChar c
    in
        head $ BSU.toString $ BS.pack [219 - cb]
              | otherwise = c

decryptChar :: Char -> Char
decryptChar c =
    if check c then
        head $ BSU.toString $ BS.pack [219 - head (BS.unpack $ BSU.fromChar c)]
    else
        c
    where
        check c = 
            let
                target = head (BS.unpack $ BSU.fromChar c) 
                min = head $ BS.unpack $ BSU.fromChar 'a'
                max = head $ BS.unpack $ BSU.fromChar 'z'
            in 
                min <= target && target <= max


encrypt :: T.Text -> T.Text
encrypt = T.map encryptChar

decrypt :: T.Text -> T.Text
decrypt = T.map decryptChar

message = "こんにちは this is encrypted message. THIS IS NOT ENCRYPTED"
putStr $ T.unpack $ encrypt message
putStr $ T.unpack $ decrypt (encrypt message) 


こんにちは gsrh rh vmxibkgvw nvhhztv. THIS IS NOT ENCRYPTED

こんにちは this is encrypted message. THIS IS NOT ENCRYPTED

In [ ]:
-- 09
import qualified System.Random as RD
import Debug.Trace

factorial :: Int -> Int
factorial 0 = 1
factorial n | n > 0 = let n' = factorial (n -1) in seq n' $ n * n' 

interleave :: a -> [a] -> [[a]]
interleave x [] = [[x]]
interleave x (y:ys) = (x:y:ys) : map (y:) (interleave x ys)

perms :: [a] -> [[a]]
perms = foldr (concatMap . interleave) [[]]

getTypoglycemia :: RD.RandomGen g => g -> String -> String
getTypoglycemia g t | length t > 4 = 
    let
        seed = perms (tail (init t))
        n = fst $ RD.randomR (0, factorial (length t - 2) -1) g
    in
        [head t] ++ (seed !! n) ++ [last t]
                    | otherwise = t

arg = "I couldn’t believe that I could actually understand what I was reading : the phenomenal power of the human mind ."

do 
    g <- RD.newStdGen
    putStrLn $ unwords $map (getTypoglycemia g) (words arg)


I cdoun’lt bvieele that I cuold aactlluy utnseadnrd what I was rndeiag : the pehmnneaol pwoer of the hmuan mind .

### chapter 2

In [30]:
getPopularNames :: IO T.Text = T.pack <$> readFile ".data/popular-names.txt"

In [20]:
-- 10

do 
    content <- readFile ".data/popular-names.txt"
    putStrLn $ show $ foldr (\c n -> if c == '\n' then 1 + n else n) 1 content 

Line 5: Use print
Found:
putStrLn
  $ show $ foldr (\ c n -> if c == '\n' then 1 + n else n) 1 content
Why not:
print (foldr (\ c n -> if c == '\n' then 1 + n else n) 1 content)

2781

-- bash
$  wc ./data/popular-names.txt

In [26]:
-- 11
do 
    content <- readFile ".data/popular-names.txt"
    putStrLn "----- input"
    putStrLn $ unlines $ take 10 $ lines content
    putStrLn "----- output"
    putStrLn $ unlines $ take 10 $ lines $ map (\c -> if c == '\t' then ' ' else c) content

----- input
Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
Minnie	F	1746	1880
Margaret	F	1578	1880
Ida	F	1472	1880
Alice	F	1414	1880
Bertha	F	1320	1880
Sarah	F	1288	1880

----- output
Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880

-- bash 
cat ./.data/popular-names.txt | tr '\t' ' '

In [43]:
-- 12
import System.IO
col1 :: String  = "./.data/out/col1.txt"
col2 :: String = "./.data/out/col2.txt"
do
    content <- T.map (\c -> if c == '\t' then ' ' else c) <$> getPopularNames
    writeFile col1 $ T.unpack $ T.unlines $ map (head . T.words) $ T.lines content
    writeFile col2 $ T.unpack $ T.unlines $ map (\line -> T.words line !! 1) $ T.lines content 

-- bash
cut --fields=1 ./.data/popular-names.txt > ./.data/out_bash/col1txt
cut --fields=2 ./.data/popular-names.txt > ./.data/out_bash/col2.txt

In [ ]:
-- 13
getCol1 = T.pack <$> readFile "./.data/out/col1.txt"
getCol2 = T.pack <$> readFile "./.data/out/col2.txt"

